In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser


llm = ChatOpenAI(temperature=0)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a very knowledgeable historian who provides accurate and eloquent answers to historical questions."),
    ("human", "{question}"),
])

from langchain.chains import LLMChain

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    output_parser=StrOutputParser()
)
chain.run(question="How are you?")

"Thank you for asking! As an AI language model, I don't have feelings, but I'm here and ready to assist you with any historical questions you may have. How can I help you today?"

In [ ]:
## Initialize from string
template = """Tell me a {adjective} joke about {subject}."""
llm_chain = LLMChain.from_string(llm=llm, template=template)
llm_chain.predict(adjective="sad", subject="ducks")

In [ ]:
## __call__ method
## By default, it returns both the input and output key values.

from langchain.prompts import PromptTemplate

chat = ChatOpenAI(temperature=0)
prompt_template = "Tell me a {adjective} joke"
llm_chain = LLMChain(
    llm=chat,
    prompt=PromptTemplate.from_template(prompt_template)
)

print(llm_chain(inputs={"adjective": "corny"}))

## add return_only_outputs=True
## Also, in the case of one input key, you can input the string directly without specifying the input mapping.
llm_chain = LLMChain(
    llm=chat,
    prompt=PromptTemplate.from_template(prompt_template)
)
print(llm_chain("corny", return_only_outputs=True))

## run method
# IF only one output_keys, you can use run method
print(llm_chain.output_keys)
print(llm_chain.run({"adjective":"corny"}))
print("run: \n", llm_chain.run("corny"))

## predict method
# similar to run, but the input_keys are specified as keyword arguments instead of a python dict
# print("predict: \n", llm_chain.predict(adjective="corney"))
# multiple inputs example
template = """Tell me a {adjective} joke about {subject}."""
prompt = PromptTemplate(template=template, input_variables=["adjective", "subject"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
print("predict: \n", llm_chain.predict(adjective="sad", subject="ducks"))

## apply method
## allows you run the chain against a list of tuples
prompt_template = "What is a good name for a company that makes {product}?"
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
input_list = [{"product": "socks"}, {"product": "computer"}, {"product": "shoes"}]
print("apply: \n", llm_chain.apply(input_list))

## generate method
## similar to apply, except it return an LLMResult
print("generate: \n", llm_chain.generate(input_list))

In [18]:
## add output_parser to prompt_template
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all the colors in a rainbow"""
prompt = PromptTemplate(
    template=template, input_variables=[]
)
llm_chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)
llm_chain.predict()

['The colors in a rainbow are:\n\n1. Red\n2. Orange\n3. Yellow\n4. Green\n5. Blue\n6. Indigo\n7. Violet']

In [11]:
## Custom Chain

from __future__ import annotations

from typing import Any, Dict, List, Optional

from pydantic import Extra

from langchain.schema.language_model import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate


class MyCustomChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)

        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            run_manager.on_text("Log something about this run")

        return {self.output_key: response.generations[0][0].text}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)

        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            await run_manager.on_text("Log something about this run")

        return {self.output_key: response.generations[0][0].text}

    @property
    def _chain_type(self) -> str:
        return "my_custom_chain"
    
from langchain.callbacks.stdout import StdOutCallbackHandler
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate

chain = MyCustomChain(
    prompt=PromptTemplate.from_template("tell us a joke about {topic}"),
    llm=ChatOpenAI(),
)

chain.run({"topic": "callbacks"}, callbacks=[StdOutCallbackHandler()])



> Entering new MyCustomChain chain...
Log something about this run
> Finished chain.


'Why did the function go to therapy?\n\nBecause it had too many unresolved callbacks!'

In [12]:
## Adding memory

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

conversation.run("Answer briefly. What are the first 3 colors of a rainbow?")
# -> The first three colors of a rainbow are red, orange, and yellow.
conversation.run("And the next 4?")
# -> The next four colors of a rainbow are green, blue, indigo, and violet.

'The next four colors of a rainbow are green, blue, indigo, and violet.'

In [1]:
import logging
import asyncio
from typing import Any, Dict, List, Optional
from functools import partial
from langchain.schema import BaseRetriever, Document
from langchain.callbacks.manager import CallbackManagerForRetrieverRun, AsyncCallbackManagerForRetrieverRun

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.DEBUG,
    datefmt="%m%d%Y %I:%M:%S %p"
)

class IRRetriever(BaseRetriever):
    """IRRetriever to retrieve documents using IR."""
    
    ir_model: Any
    """IR vectorizer"""
    topk: int=5
    """Number of documents to return"""
    search_args: Dict=None
    """Search arguments of IR"""
    retrieved_docs: List[Document]=None
    """List of documents retrieved"""
    
    class Config:
        """Configuration for this pydantic object"""
        
        arbitrary_types_allowed = True
    
    
    def check_is_project_exist(self, project_name, where):
        projects = self.ir_model.list_projects()[where]
        is_exist = False
        for project in projects:
            name = project.get("name")
            if project_name == name:
                is_exist = True
        return is_exist
    
    def create_new_project(self, project_name, where, index_args=None):
        if index_args is None:
            index_args = self.ir_model.get_default_index_args()
        
        self.ir_model.new_project(
            project_name=project_name, where=where, index_args=index_args
        )
        logging.info("New project named `{}` created successfully.")

    def load_project(self, project_name, where):
        is_exist = self.check_is_project_exist(project_name, where)
        if not is_exist:
            raise ValueError(
                f"{project_name} not found in {where}. Please create the project first."
            )
        else:
            # load project -> get self.search_args
            self.search_args = self.ir_model.load_project(
                project_name=project_name, where=where
            )
            logging.info("Load project succesfully.")
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """Get documents relevant to a query.
        Args:
            query: String to find relevant documents for
            run_manager: The callbacks handler to use
        Returns:
            List of relevant documents
        """
        if self.search_args is None:
            raise Exception("No project loaded. Please use `load_project` method to load project in advance.")
        
        self.search_args["tok_k"] = self.topk
        docs = self.ir_model.search(query, search_args=self.search_args)
        
        self.retrieved_docs = []
        for i, doc in enumerate(docs, start=1):
            title, context = doc.pop("title"), doc.pop("context")
            page_content = f"{str(i)}. {title}\n{context}"
            self.retrieved_docs += [
                Document(
                    page_content=page_content,
                    meta_data=doc,
                    type="Document"
                )
            ]
        return self.retrieved_docs
    
    async def _aget_relevant_documents(
        self, query: str, *, run_manager: AsyncCallbackManagerForRetrieverRun
    ) -> List[Document]:
        """Asynchronously get documents relevant to a query.
        Args:
            query: String to find relevant documents for
            run_manager: The callbacks handler to use
        Returns:
            List of relevant documents
        """
        return await asyncio.get_running_loop().run_in_executor(
            None, partial(self._get_relevant_documents, run_manager=run_manager), query
        )

In [2]:
from __future__ import annotations
import inspect
from typing import Any, Dict, List, Optional

from pydantic import Extra

from langchain.schema.language_model import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate


class IRChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    retriever: BaseRetriever
    output_key: str = "answer"
    return_source_documents: bool = True
    """Return the retrieved source documents as part of the final result."""
    response_if_no_docs_found: Optional[str]
    """If specified, the chain will return a fixed response if no docs 
    are found for the question. """

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True
        allow_population_by_field_name = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects."""
        
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Return the output keys."""
        _output_keys = [self.output_key]
        if self.return_source_documents:
            _output_keys = _output_keys + ["source_documents"]
        return _output_keys
    
    def _get_docs(
        self,
        question: str,
        inputs: Dict[str, Any],
        *,
        run_manager: CallbackManagerForChainRun,
    ) -> List[Document]:
        """Get relevant documents."""
        docs = self.retriever.get_relevant_documents(
            question, callbacks=run_manager.get_child()
        )
        return docs
    
    async def _aget_docs(
        self,
        question: str,
        inputs: Dict[str, Any],
        *,
        run_manager: AsyncCallbackManagerForChainRun
    ) -> List[Document]:
        """Get relevant documents."""
        docs = await self.retriever.aget_relevant_documents(
            question, callbacks=run_manager.get_child()
        )
        return docs

    def create_document_scratchpad(self, docs: List[Document]) -> str:
        document_scratchpad = ""
        for doc in docs:
            document_scratchpad += f"""{doc.page_content}\n\n"""
        return document_scratchpad
    
    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        question = inputs["question"]
        
        accepts_run_manager = (
            "run_manager" in inspect.signature(self._get_docs).parameters
        )
        if accepts_run_manager:
            docs = self._get_docs(question, inputs, run_manager=_run_manager)
        else:
            docs = self._get_docs(question, inputs)
            
        document_scratchpad = self.create_document_scratchpad(docs)
        
        output: Dict[str, Any] = {}
        if self.response_if_no_docs_found is not None and len(docs) == 0:
            output[self.output_key] = self.response_if_no_docs_found
        else:
            new_inputs = inputs.copy()
            new_inputs["document_scratchpad"] = document_scratchpad
            
            prompt_value = self.prompt.format_prompt(**inputs)
            answer = self.llm.generate_prompt(
                [prompt_value], callbacks=_run_manager.get_child()
            )
            output[self.output_key] = answer
        
        if self.return_source_documents:
            output["source_documents"] = docs
        
        if _run_manager:
            _run_manager.on_text("IRChain finally runs!!!")

        return output

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        question = inputs["question"]
        
        accepts_run_manager = (
            "run_manager" in inspect.signature(self._get_docs).parameters
        )
        if accepts_run_manager:
            docs = await self._aget_docs(question, inputs, run_manager=_run_manager)
        else:
            docs = await self._aget_docs(question, inputs)
            
        document_scratchpad = self.create_document_scratchpad(docs)
        
        output: Dict[str, Any] = {}
        if self.response_if_no_docs_found is not None and len(docs) == 0:
            output[self.output_key] = self.response_if_no_docs_found
        else:
            new_inputs = inputs.copy()
            new_inputs["document_scratchpad"] = document_scratchpad
            
            prompt_value = self.prompt.format_prompt(**inputs)
            answer = await self.llm.agenerate_prompt(
                [prompt_value], callbacks=_run_manager.get_child()
            )
            output[self.output_key] = answer
        
        if self.return_source_documents:
            output["source_documents"] = docs
        
        if _run_manager:
            _run_manager.on_text("IRChain finally runs!!!")

        return output

    @property
    def _chain_type(self) -> str:
        return "IR_Chain"